# 결측치 처리방식에 따라 달라지는 분류기의 성능 확인


## 1. 결측이 발생한 행, 열정보를 삭제하는 방법
> - dropna를 활용한 결측치가 존재하는 행 또는 열정보 제거
>   - df.dropna(axis=0) # axis=0 열, axis=1 행

## 2. 특정 값을 활용한 결측치 처리 방법
2-1. fillna(값)를 통한 특정 값으로 채우기
> - 0으로 채우기: df.fillna(0)
> - 평균 : df.fillna(df.mean())
> - 중앙값 : df.fillna(df.median())
> - 최빈값 : df.fillna(df.mode())
> - 이전행의 값으로 채우기: df.fillna(method='ffill')
> - 바로 다음행의 값으로 채우기: df.fillna(method='bfill')

2-2. 그룹연산(groupby)으로 분류된 각 그룹 단위의 평균값을 활용하여 채우기
> ```
> fill_mean_grp = lambda g: g.fillna(g.mean())
> df = df.groupby('그룹조건컬럼명').apply(fill_mean_grp)
> ```

2-3. 간단한 선형비례를 이용하여 대체하는 방법
> - df = df.interpolate(method='values')


## 3. 다른 알고리즘를 활용한 결측치 처리 방법
- KNN을 활용하여 유사한 패턴을 보이는 데이터의 값을 참고하여 대체 <br>
  이때 column 및 index값이 숫자로 변경됨
> ```
> imputer = KNNImputer(n_neighbors=3) # n_neighbors: 결측치 처리에 참고할 이웃값 수
> knn_data = pd.DataFrame(imputer.fit_transform(df))
> ```



# 1. 결측이 발생한 행 또는 열정보 제거: dropna(axis=축방향)
- axis=0 # 열 제거
- axis=1 # 행 제거


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

In [ ]:
# 데이터셋 결측치 수 확인
married_dataset.isnull().sum()

In [ ]:
# 항목별 결측치 비율 확인
married_dataset.isna().mean()

In [ ]:
# 결측치를 제거하여 처리
married_dataset = married_dataset.dropna(axis=0)
married_dataset

In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

x

In [ ]:
from sklearn.model_selection import train_test_split

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

In [ ]:
# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

In [ ]:
print(confusion_matrix(y_test, pred))  # confusion matrix

In [ ]:
print(classification_report(y_test, pred))  # classification repor

# 2. 특정 값을 활용한 결측치 처리

## 2-1. fillna(값)를 통한 특정 값으로 채우기
> - 0으로 채우기: df.fillna(0)
> - 평균 : df.fillna(df.mean())
> - 중앙값 : df.fillna(df.median())
> - 최빈값 : df.fillna(df.mode())
> - 이전행의 값으로 채우기: df.fillna(method='ffill')
> - 바로 다음행의 값으로 채우기: df.fillna(method='bfill')

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)
married_dataset

In [ ]:
'''
0으로 채우기: df.fillna(0)
평균 : df.fillna(df.mean())
중앙값 : df.fillna(df.median())
최빈값 : df.fillna(df.mode())
이전행의 값으로 채우기: df.fillna(method='ffill')
바로 다음행의 값으로 채우기: df.fillna(method='bfill')
'''

married_dataset = married_dataset.fillna(married_dataset.median()) #변경 가능

married_dataset

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

## 2-2. 그룹연산(groupby)으로 분류된 각 그룹 단위의 평균값을 활용한 결측치 처리
> ```
> fill_mean_grp = lambda g: g.fillna(g.mean())
> df = df.groupby('그룹조건컬럼명').apply(fill_mean_grp)
> ```

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

In [ ]:
# 결혼 성공 여부에 따라 그룹화 하여 해당 그룹내 각 컬럼별 평균값으로 결측치 처리
fill_mean_grp = lambda g: g.fillna(g.mean())
married_grp_mean = married_dataset.groupby('married').apply(fill_mean_grp)

married_grp_mean

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_grp_mean.drop(['married'], axis=1)
y = married_grp_mean['married']

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

## 2-3. 간단한 선형비례를 이용하여 대체하는 방법

df = df.interpolate(method='values')

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

In [ ]:
married_dataset = married_dataset.interpolate(method='values')

married_dataset

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train_test_split를 활용한 train set, test set 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
# 예시로 xgboost 활용
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

# 테스트 데이터로 예측
pred = model.predict(X_test)

accuracy_score(y_test, pred)  # accuracy

# 3. 다른 알고리즘를 활용한 결측치 처리 방법: KNN


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

#한번에 생략없이 출력하고 싶은 컬럼 수 설정
#pd.options.display.max_columns = 25

married_dataset

In [ ]:
# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

In [ ]:
# 사이킷런 라이브러리의 KNNImputer 불러오기
from sklearn.impute import KNNImputer

# KNNImputer 객체 생성과 KNN알고리즘에서 중요한 n_neighbors 수(참고할 이웃값 수) 설정
imputer = KNNImputer(n_neighbors=3)

married_knn = pd.DataFrame(imputer.fit_transform(married_dataset))
married_knn

In [ ]:
# 컬럼명이 숫자로 변환되었으므로 다시 원래의 이름으로 변환
married_knn.columns = married_dataset.columns

married_knn

In [ ]:
from sklearn.model_selection import train_test_split

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_knn.drop(['married'], axis=1)
y = married_knn['married']

# train set 과 test set 으로 데이터를 나누기 위해 train_test_split 활용
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import xgboost as xgb

# model = xgb.XGBClassifier(n_estimators=500, max_depth=5, random_state=100)
model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

pred = model.predict(X_test)

print('accuracy:',accuracy_score(y_test, pred))
print('f1-score:',f1_score(y_test,pred))

In [ ]:
print(confusion_matrix(y_test, pred))  # confusion matrix 출력

In [ ]:
print(classification_report(y_test, pred))  # classification report 출력

# 하이퍼파라미터 튜닝

- optuna


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 데이터 불러오기
# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train set 과 test set 으로 데이터를 나누기 위해 train_test_split 함수를 불러옴
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(random_state=100)
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

pred = model.predict(X_test)

print('accuracy:',accuracy_score(y_test, pred))
print('f1-score:',f1_score(y_test,pred))

### XGBoost Classifier에 있는 Hyperparameter
* XGBoost API 설명자료: https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn  

* 코드에서 활용한 하이퍼 파라미터
> * objective: 모델의 학습 과제 및 학습목표(일반적으로 classifier의 경우 이진 혹은 다중 분류로 기본값이 설정되어 있음)
> * num_leaves: tree의 최대 leaf 수
> * learning_rate: 학습률
> * n_estimators: 학습에 사용할 트리 수(xgb학습과정에서 부스팅 라운드 수)
> * max_depth: 트리의 최대 깊이
> * ramdom_state: 결과 재현을 위한 시드값


In [ ]:
#현재 코랩 런타임에 Optuna 라이브러리 설치
!pip install optuna

In [ ]:
import optuna

def objective(trial):
    global X_train, X_test, y_train, y_test
    xbg_trainset = xgb.DMatrix(X_train, label=y_train)
    xgb_testset = xgb.DMatrix(X_test, label=y_test)

    # 최적화할 하이퍼 파라미터 지정 및 찾아볼 값 범위 설정
    param = {
        "objective": "binary:logistic",
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 0.3),
        'n_estimators':  trial.suggest_int('n_estimators',100,3000 ),
        'max_depth': trial.suggest_int("max_depth", 3, 21, step=2),
        'random_state': 100,
    }

    model_train = xgb.train(param, xbg_trainset)
    preds = model_train.predict(xgb_testset)
    pred_labels = np.rint(preds)

    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy

# optuna에서의 최적화 할 study 생성과 최적화 방향(지표의 값을 최대화 할 것인지)
study = optuna.create_study(direction="maximize")

# trial 횟수 설정 및 최적화 시작
study.optimize(objective, n_trials=500, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

In [ ]:
# best trial에 대한 성능과 hyperparameter 정보 출력
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
#plot_optimization_histor: trial 진행과정 히스토리
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: 하이퍼파라미터의 조합과 점수에 대한 시각화
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
#최적화 과정에서 계산된 하이퍼 파라미터 별 성능에 영향을 미친 중요도 시각화
optuna.visualization.plot_param_importances(study)

# [추가실습자료] LightGBM 알고리즘 활용하기 + optuna

In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 출력 표기 생략
warnings.filterwarnings('ignore')

# 파일 경로 = "https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv"
married_dataset = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")

# 결측치 처리
married_dataset = married_dataset.dropna(axis=0)

# object 타입의 문자열 변수를 숫자형으로 변환
married_dataset = pd.get_dummies(married_dataset, columns=['gender'], drop_first=True)

# 데이터셋, 독립변수와 종속변수 분리: 독립변수 -> x, 종속변수 -> y
x = married_dataset.drop(['married'], axis=1)
y = married_dataset['married']

# train set 과 test set 으로 데이터를 나누기 위해 train_test_split 함수를 불러옴
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=100)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
import lightgbm as lgb

model = lgb.LGBMClassifier(random_state=100)
model.fit(X_train, y_train)

pred = model.predict(X_test)  # 테스트 데이터로 예측

print('accuracy:',accuracy_score(y_test, pred))
print('f1-score:',f1_score(y_test,pred))

In [ ]:
print(confusion_matrix(y_test, pred))  # confusion matrix 출력

In [ ]:
print(classification_report(y_test, pred))  # classification report 출력

### LightGBM Classifier에 있는 Hyperparameter
- LightGBMClassifier : https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier
- 코드에서 활용한 하이퍼파라미터
> * objective: 모델의 학습 과제 및 학습목표(일반적으로 classifier의 경우 이진 혹은 다중 분류로 기본값이 설정되어 있음)
> * num_leaves: tree의 최대 leaf 수
> * learning_rate: 학습률
> * n_estimators: 학습에 사용할 트리 수
> * max_depth: 트리의 최대 깊이
> * ramdom_state: 결과 재현을 위한 시드값

- 아래 문구가 학습시 나오더라도 오류는 아니니 무시할 것(더 분할할 수 없어서 나타나는 경고)

  [Warning] No further splits with positive gain, best gain: -inf

In [ ]:
# Optuna
import optuna
def objective(trial):
    global X_train, X_test, y_train, y_test

    # 최적화 할 하이퍼파라미터 지정 및 값의 범위 설정
    param = {
        "objective": "binary",
        'num_leaves': trial.suggest_int('num_leaves', 2, 512, step=1, log=True),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 0.3),
        'n_estimators':  trial.suggest_int('n_estimators',100,1000 ),
        'max_depth': trial.suggest_int("max_depth", 3, 21, step=2),
        'random_state': 100,
    }

    model = lgb.LGBMRegressor(**param)
    lgb_model = model.fit(X_train, y_train, eval_set=[(X_test, y_test)])
    preds = lgb_model.predict(X_test)
    pred_labels = np.rint(preds)

    accuracy = accuracy_score(y_test, pred_labels)
    return accuracy


# optuna에서의 최적화 할 study 생성과 최적화 방향(지표의 값을 최대화 할 것인지)
study = optuna.create_study(direction="maximize")

# trial 횟수 설정 및 최적화 시작
study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: ", len(study.trials))

In [ ]:
# best trial에 대한 성능과 hyperparameter 정보 출력
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
#plot_optimization_history
optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
#Visualize parameter importances.
optuna.visualization.plot_param_importances(study)